In [14]:
def readDataFrameFromCloudant(host,user,pw,database):
    sparkSession = SQLContext.getOrCreate(sc).sparkSession

    cloudantdata=sparkSession.read.format("com.cloudant.spark"). \
    option("cloudant.host",host). \
    option("cloudant.username", user). \
    option("cloudant.password", pw). \
    load(database)

    cloudantdata.createOrReplaceTempView("washing")
    #spark.sql("SELECT * from washing").show()
    return cloudantdata

In [15]:
#TODO Please provide your Cloudant credentials here
hostname = "pleasePutYourCredentialsHere"
user = "pleasePutYourCredentialsHere"
pw = "pleasePutYourCredentialsHere"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same
cloudantdata=readDataFrameFromCloudant(hostname, user, pw, database)

In [16]:
#spark.sql("select avg(voltage), std(voltage), max(voltage) from washing").show()

In [17]:
df = spark.sql("""
select mean(voltage) as mean, std(voltage) as std from (
    SELECT ROW_NUMBER() OVER (ORDER BY ts desc) AS row, * FROM washing where voltage is not null
) where
row < 30
""")
               
#df.show()
mean = df.first().mean
std = df.first().std
#spark.sql("SELECT * from (select abs( ({0}-voltage)/({1}+0.0001) ) as zscore, voltage,ROW_NUMBER() OVER (ORDER BY ts desc) AS row FROM washing where voltage is not null order by ts desc) where row < 20".format(mean,std)).show()
df = spark.sql("SELECT mean(zscore) as avgzscore from (select abs( ({0}-voltage)/({1}+0.0001) ) as zscore, voltage,ROW_NUMBER() OVER (ORDER BY ts desc) AS row FROM washing where voltage is not null order by ts desc) where row < 20".format(mean,std))
df.show()

+--------------------+
|           avgzscore|
+--------------------+
|1.020956917482053158|
+--------------------+



In [18]:
zscore = df.first().avgzscore

In [19]:
import requests
if (zscore > 0.5):
    r = requests.get("https://1234thisismyname.mybluemix.net/edgemodelupdate?command=emergencyshutdown")
    r.status_code